# Extracting Tables and Images from PDF 

Ref -> https://stackoverflow.com/questions/2693820/extract-images-from-pdf-without-resampling-in-python

In [11]:
import pandas as pd
import PyPDF2 as p2
import camelot
import re
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from PIL import Image
# import ghostscript

#from pdf2image import convert_from_path

In [12]:
index2 = pd.read_csv('F:\Environmental Baseline Data\Version 3\Indices\Final\Index 2 - List of PDFs for Major Projects with ESAs.csv')
ngtl_2021 = index2[index2['Application title short'].str.contains("Application for 2021 NGTL System Expansion Project")]

In [ ]:
index2 = pd.read_csv('F:\Environmental Baseline Data\Version 3\Indices\Final\Index 2 - List of PDFs for Major Projects with ESAs.csv')
ngtl_2021 = index2[index2['Application title short'].str.contains("Application for 2021 NGTL System Expansion Project")]
ngtl_2021_pdfs = ngtl_2021['PDF file name'].tolist()

path = 'F:/Environmental Baseline Data/Version 3/Data/PDF/'
ngtl_2021_pdfs_path = [path + x for x in ngtl_2021_pdfs]


def get_list_elements(s, level):
    if isinstance(s, list):
        for x in s:
            get_list_elements(x, level+1)
    else:
        data.append({'Outline Item' : s.title, 'Page Number' : pdf_reader.getDestinationPageNumber(s) + 1, 'Level' : level, 'file' : y})
    return

data = []
for y in ngtl_2021_pdfs_path:
    PDFfile = open(y, 'rb')
    pdf_reader = p2.PdfFileReader(PDFfile)
    
    get_list_elements(pdf_reader.outlines, 0)
df_outlines = pd.DataFrame(data)

### Extracting Tables from the Outlines 

In [4]:
df_outlines_tables = df_outlines[df_outlines['Outline Item'].str.contains("able", na = False)]
df_outlines_tables_sample = df_outlines_tables
df_outlines_tables_sample.head(10)

,Outline Item,Page Number,Level,file
1,Master Table of Contents,3,2,F:/Environmental Baseline Data/Version 3/Data/...
10,1.5 Applicable Legislation and Additional Regu...,18,4,F:/Environmental Baseline Data/Version 3/Data/...
11,Table 1.5–1: Additional Regulatory and Environ...,19,5,F:/Environmental Baseline Data/Version 3/Data/...
14,Table 1.7–1: Concordance with Guide A – A.2 En...,21,5,F:/Environmental Baseline Data/Version 3/Data/...
16,Table 1.8–1: Concordance with Canadian Environ...,24,5,F:/Environmental Baseline Data/Version 3/Data/...
18,Table 1.9–1: Project Environmental and Socio-e...,26,5,F:/Environmental Baseline Data/Version 3/Data/...
28,Table 2.3–1: Parallel Rights-of-Way and Non-Pa...,33,5,F:/Environmental Baseline Data/Version 3/Data/...
33,Table 2.4–1: Technical Details of Pipeline Com...,36,5,F:/Environmental Baseline Data/Version 3/Data/...
36,Table 2.4–2: Details of Proposed Compressor St...,43,5,F:/Environmental Baseline Data/Version 3/Data/...
37,Table 2.4–3: Preliminary Compressor Station Lo...,44,5,F:/Environmental Baseline Data/Version 3/Data/...


In [5]:
for x, y, z, m in zip(df_outlines_tables_sample['Page Number'], 
                      df_outlines_tables_sample['Outline Item'], 
                      df_outlines_tables_sample['file'], 
                      df_outlines_tables_sample['file']):
    
    figure_name = str(y).replace(':', '_')
    file_short = str(re.search('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)', z).group(1))
    path_start = str('F:/Evironmental Baseline Data/Version 2/Data/PDF test NGTL 2021/')
    
    tables = camelot.read_pdf(z, pages = str(x), flag_size=True)  #loop len(tables)
      
    df = tables[0].df #tables may be greater than 1, add loop
    df = df.replace('/na', '_', regex = True)
    path = str('F:/Environmental Baseline Data/Version 3/PDF test NGTL 2021/')

    save_string = path + '2021 NGTL expansion_' + str(figure_name) + '-pg.' + str(x) + '-' + str(file_short) + '.csv'
    df.to_csv(save_string, index = False)
    
    print(z)

AttributeError: module 'camelot' has no attribute 'read_pdf'

### Extracting Images from the Outlines 

In [6]:
df_outlines_figures = df_outlines[df_outlines['Outline Item'].str.contains("igure", na = False)]
df_outlines_figures_sample = df_outlines_figures[0:100]
len(df_outlines_figures_sample)
df_outlines_figures_sample.head(5)

100

,Outline Item,Page Number,Level,file
8,Figure 1.3–1: 2021 NGTL System Expansion Proje...,16,5,F:/Environmental Baseline Data/Version 3/Data/...
46,Figure 2.6–1: Preliminary Construction Schedule,50,5,F:/Environmental Baseline Data/Version 3/Data/...
68,Figure 4.3–1: Local Study Area (Pipeline),71,4,F:/Environmental Baseline Data/Version 3/Data/...
69,Figure 4.3–2: Local Study Area (Compressor Sta...,72,4,F:/Environmental Baseline Data/Version 3/Data/...
70,Figure 4.3–3: Regional Study Area,73,4,F:/Environmental Baseline Data/Version 3/Data/...


In [7]:
# Making a set of file names 
file_names_set = []
for file_name in df_outlines_tables_sample['file']:
    file_names_set.append((file_name.split('/')[-1]).split('.')[0])
file_names_set = set(file_names_set)
file_names_set


{'A6F4Q3',
 'A6F4Q4',
 'A6F4Q5',
 'A6F4Q6',
 'A6F4Q7',
 'A6F4Q8',
 'A6F4Q9',
 'A6F4R2'}

In [8]:
# Making the Relevand folders for all unique File Names 
def makedirectory(dirName):
    try:
        # Create target Directory
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ") 
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")


In [9]:
# Ref-> https://thispointer.com/how-to-create-a-directory-in-python/
path = "F:\\Environmental Baseline Data\\Version 3\\Data\\Images\\"
makedirectory(path)

for file_name in file_names_set:
    makedirectory(path + file_name)

Directory  F:\Environmental Baseline Data\Version 3\Data\Images\  already exists
Directory  F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q9  already exists
Directory  F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q5  already exists
Directory  F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q4  already exists
Directory  F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q3  already exists
Directory  F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q8  already exists
Directory  F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q7  already exists
Directory  F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q6  already exists
Directory  F:\Environmental Baseline Data\Version 3\Data\Images\A6F4R2  already exists


Ref -> https://stackoverflow.com/questions/2693820/extract-images-from-pdf-without-resampling-in-python <br>
Ref -> https://gist.github.com/maphew/fe6ba4bf9ed2bc98ecf5 <br>
REf -> https://github.com/Belval/pdf2image  <br>

<br>
<br>
<br>
Best Ref -> https://pythonforundergradengineers.com/pdf-to-multiple-images.html


In [10]:
from PyPDF2 import PdfFileReader
from PythonMagick import Image
myfile = PdfFileReader(df_outlines_figures_sample.iloc[0]['file'])



pages = myfile.getNumPages()
print(pages)

ModuleNotFoundError: No module named 'PythonMagick'

In [105]:

for page in pages:
    im = Image(myfile.getPage(i+1))
    im.write('file_image{}.png'.format(i+1))

NameError: name 'pages' is not defined

In [75]:
project_name = '2021 NGTL expansion_' 




for i in range(len(df_outlines_figures_sample)):
    pdf_reader = p2.PdfFileReader(df_outlines_figures_sample.iloc[i]['file'])
    print("The PDF {} has {} number of pages".format(df_outlines_figures_sample.iloc[i]['file'], pdf_reader.getNumPages()))
    
    pageNo = df_outlines_figures_sample.iloc[i]['Page Number']
    
    file_name = (df_outlines_figures_sample.iloc[i]['file'].split('/')[-1]).split('.')[0]
    print(path + file_name + "\\")
    
    figure_name = df_outlines_figures_sample.iloc[i]['Outline Item']
    figure_name = re.sub('[^A-Za-z0-9]+', ' ', figure_name)
    
    page_no = df_outlines_figures_sample.iloc[i]['Page Number']
    
    
    image_name = path + file_name + "\\" + str(figure_name) + '-pg-' + str(pageNo) + '-' + str(file_name)
    
    
    
    
    
    
"""    #image_name + "-pg-" + page_no + 
    
    #os.chdir(r(path + file_name + "\\"))
    # extracting images from the page
    page0 = pdf_reader.getPage(pageNo).rotateClockwise(90)
    if '/XObject' in page0['/Resources']:
        xObject = page0['/Resources']['/XObject'].getObject()

        for obj in xObject:
            if xObject[obj]['/Subtype'] == '/Image':
                size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                data = xObject[obj].getData()
                if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                    mode = "RGB"
                else:
                    mode = "P"

                if '/Filter' in xObject[obj]:
                    if xObject[obj]['/Filter'] == '/FlateDecode':
                        img = Image.frombytes(mode, size, data)
                        img = img.transpose(Image.ROTATE_90)
                        img.save(image_name+ obj[1:] + ".png")
                        print('/FlateDecode')
                        
                    elif xObject[obj]['/Filter'] == '/DCTDecode':
                        img = open(image_name+ obj[1:] +".jpg", "wb")
                        img = img.transpose(Image.ROTATE_90)
                        img.write(data)
                        img.close()
                        print('/DCTDecode')
                        
                    elif xObject[obj]['/Filter'] == '/JPXDecode':
                        img = open(image_name+ obj[1:] + ".jp2", "wb")
                        img = img.transpose(Image.ROTATE_90)
                        img.write(data)
                        img.close()
                        print('/JPXDecode')
                        
                    elif xObject[obj]['/Filter'] == '/CCITTFaxDecode':
                        img = open(image_name+ obj[1:] + ".tiff", "wb")
                        img = img.transpose(Image.ROTATE_90)
                        img.write(data)
                        img.close()
                        print('/CCITTFaxDecode')
                else:
                    img = Image.frombytes(mode, size, data)
                    img = img.transpose(Image.ROTATE_90)
                    img.save(image_name+ obj[1:] + ".png")
    else:
        print("No image found.")"""

The PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf has 112 number of pages
F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q3\
/FlateDecode
The PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf has 112 number of pages
F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q3\
/FlateDecode
The PDF F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf has 112 number of pages
F:\Environmental Baseline Data\Version 3\Data\Images\A6F4Q3\
/FlateDecode


AttributeError: '_io.BufferedWriter' object has no attribute 'transpose'

In [76]:
type(img)
img

_io.BufferedWriter

<_io.BufferedWriter name='F:\\Environmental Baseline Data\\Version 3\\Data\\Images\\A6F4Q3\\Figure 4 3 1 Local Study Area Pipeline -pg-71-A6F4Q3Im1.jpg'>

In [32]:
cwd = os.getcwd() 
type(cwd)
cwd


str

'C:\\Users\\T1Vibudh\\Desktop'

In [ ]:
      save_string = path + '2021 NGTL expansion_' + str(figure_name) + '-pg.' + str(x) + '-' + str(file_short) + '.csv'
    df.to_csv(save_string, index = False)

In [11]:
file_name = 'F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4R2.pdf'
input1 = p2.PdfFileReader(open(file_name, "rb"))
page0 = input1.getPage(0)
xObject = page0['/Resources']['/xObject'].getObject()

for obj in xObject:
    if xObject[obj]['/Subtype'] == '/Image':
        size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
        data = xObject[obj].getData()
        if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
            mode = "RGB"
        else:
            mode = "P"

        if xObject[obj]['/Filter'] == '/FlateDecode':
            img = Image.frombytes(mode, size, data)
            img.save(obj[1:] + ".png")
        elif xObject[obj]['/Filter'] == '/DCTDecode':
            img = open(obj[1:] + ".jpg", "wb")
            img.write(data)
            img.close()
        elif xObject[obj]['/Filter'] == '/JPXDecode':
            img = open(obj[1:] + ".jp2", "wb")
            img.write(data)
            img.close()

KeyError: '/xObject'